# CS 105 Mini Project

**Team**: Thomas Henningson, Josh Pennington, Suhail-Anand Dhilon, Cameron Tsai <br><br>
This is our CS 105 Mini Project.

# Introduction

**Question 1**: What data do you have?

1. What is your cumulative GPA as of Winter 2022?
2. What was your GPA your very first quarter at UCR?
3. What is your gender?
4. What is your age?
5. How many STEM advanced classes did you take in high school (AP, IB, Honors, Dual-Enrollment, etc.)?
6. How many social studies/arts advanced classes did you take in high school (AP, IB, Honors, Dual-Enrollment, etc.)?
7. How often do you procrastinate?
8. Rate the importance of your education in your life on a scale of 1-10
9. Rate the enjoyment/fufillment you receive from education on a scale of 1-10
10. How many hours do you study per week on average (not including homework)?

**Question 2**: What would you like to know?

Does taking advanced courses in high school have a postive effect on student's grades, study habits, and outlook on education in college?

# Exploring the Data

uhh

# Hypothesis

uhh

# Testing

uhh